In [ ]:
from __future__ import print_function

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from PIL import Image
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
import torchvision.models as models

import copy

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# desired size of the output image
imsize = (512, 512) # Here we tried several sizes which highly affected the output results

loader = transforms.Compose([
    transforms.Resize(imsize),  # scale imported image
    transforms.ToTensor()])  # transform it into a torch tensor


def image_loader(image_name):
    image = Image.open(image_name)
    image = image.convert('RGB')
    print(image.size)
    # fake batch dimension required to fit network's input dimensions
    image = loader(image).unsqueeze(0)
    print(image.shape)
    return image.to(device, torch.float)

In [ ]:
class ContentLoss(nn.Module):

    def __init__(self, target,):
        super(ContentLoss, self).__init__()
        # we 'detach' the target content from the tree used
        # to dynamically compute the gradient: this is a stated value,
        # not a variable. Otherwise the forward method of the criterion
        # will throw an error.
        self.target = target.detach()

    def forward(self, input):
        self.loss = F.mse_loss(input, self.target)
        return input

In [ ]:
def gram_matrix(input):
    a, b, c, d = input.size()  # a=batch size(=1)
    # b=number of feature maps
    # (c,d)=dimensions of a f. map (N=c*d)

    features = input.view(a * b, c * d)  # resise F_XL into \hat F_XL

    G = torch.mm(features, features.t())  # compute the gram product

    # we 'normalize' the values of the gram matrix
    # by dividing by the number of element in each feature maps.
    return G.div(a * b * c * d)

In [ ]:
class StyleLoss(nn.Module):

    def __init__(self, target_feature):
        super(StyleLoss, self).__init__()
        self.target = gram_matrix(target_feature).detach()

    def forward(self, input):
        G = gram_matrix(input)
        self.loss = F.mse_loss(G, self.target)
        return input

In [ ]:
cnn = models.vgg19(pretrained=True).features.to(device).eval()

In [ ]:
cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
cnn_normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(device)

# create a module to normalize input image so we can easily put it in a
# nn.Sequential
class Normalization(nn.Module):
    def __init__(self, mean, std):
        super(Normalization, self).__init__()
        # .view the mean and std to make them [C x 1 x 1] so that they can
        # directly work with image Tensor of shape [B x C x H x W].
        # B is batch size. C is number of channels. H is height and W is width.
        self.mean = torch.tensor(mean).view(-1, 1, 1)
        self.std = torch.tensor(std).view(-1, 1, 1)

    def forward(self, img):
        # normalize img
        return (img - self.mean) / self.std

In [ ]:
# desired depth layers to compute style/content losses :
content_layers_default = ['conv_5']

style_layers_default = None # just an initialization, we will be feeding this to the "get_style_model_and_losses" function

def get_style_model_and_losses(cnn, normalization_mean, normalization_std,
                               style_img, content_img,
                               content_layers=content_layers_default,
                               style_layers=style_layers_default):
    # normalization module
    normalization = Normalization(normalization_mean, normalization_std).to(device)

    # just in order to have an iterable access to or list of content/syle
    # losses
    content_losses = []
    style_losses = []

    # assuming that cnn is a nn.Sequential, so we make a new nn.Sequential
    # to put in modules that are supposed to be activated sequentially
    model = nn.Sequential(normalization)

    i = 0  # increment every time we see a conv
    for layer in cnn.children():
        if isinstance(layer, nn.Conv2d):
            i += 1
            name = 'conv_{}'.format(i)
        elif isinstance(layer, nn.ReLU):
            name = 'relu_{}'.format(i)
            # The in-place version doesn't play very nicely with the ContentLoss
            # and StyleLoss we insert below. So we replace with out-of-place
            # ones here.
            layer = nn.ReLU(inplace=False)
        elif isinstance(layer, nn.MaxPool2d):
            name = 'pool_{}'.format(i)
        elif isinstance(layer, nn.BatchNorm2d):
            name = 'bn_{}'.format(i)
        else:
            raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))

        model.add_module(name, layer)

        if name in content_layers:
            # add content loss:
            target = model(content_img).detach()
            content_loss = ContentLoss(target)
            model.add_module("content_loss_{}".format(i), content_loss)
            content_losses.append(content_loss)

        if name in style_layers:
            # add style loss:
            target_feature = model(style_img).detach()
            style_loss = StyleLoss(target_feature)
            model.add_module("style_loss_{}".format(i), style_loss)
            style_losses.append(style_loss)

    # now we trim off the layers after the last content and style losses
    for i in range(len(model) - 1, -1, -1):
        if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
            break

    model = model[:(i + 1)]

    return model, style_losses, content_losses

In [ ]:
def get_input_optimizer(input_img):
    # this line to show that input is a parameter that requires a gradient
    optimizer = optim.LBFGS([input_img])
    return optimizer

In [ ]:
def run_style_transfer(cnn, normalization_mean, normalization_std,
                       content_img, style_img1, p1, style_img2, input_img, num_steps=300,
                       style_weight=1000000, content_weight=1):
    """Run the style transfer."""
    print('Building the style transfer model..')
    model1, style_losses1, content_losses1 = get_style_model_and_losses(cnn,
        normalization_mean, normalization_std, style_img1, content_img, style_layers=style_layers_default1)
    model2, style_losses2, content_losses1 = get_style_model_and_losses(cnn,
        normalization_mean, normalization_std, style_img2, content_img, style_layers=style_layers_default2)
    
#     style_losses1 = [0.5*i for i in style_losses1]  # here we tried weighted the losses but we figured a better place for that
#     style_losses2 = [0.5*i for i in style_losses2]
#     style_losses = 0.style_losses1 + 0.style_losses2
    content_losses = content_losses1
    
#     model = model2
    
    # We want to optimize the input and not the model parameters so we
    # update all the requires_grad fields accordingly
    input_img.requires_grad_(True)
    model1.requires_grad_(False)
    model2.requires_grad_(False) # here we are using 2 models so that we can make the novel dual style transfer

    optimizer = get_input_optimizer(input_img)

    print('Optimizing..')
    run = [0]
    while run[0] <= num_steps:

        def closure():
            # correct the values of updated input image
            with torch.no_grad():
                input_img.clamp_(0, 1)

            optimizer.zero_grad()
            model1(input_img)
            model2(input_img)
            style_score = 0
            content_score = 0
            # here we are using 2 style losses to make sure we get to the result were the input image get the style of the 2 images
            for sl in style_losses1:
                style_score += p1*sl.loss
            for sl in style_losses2:
                style_score += (1-p1)*sl.loss
                
            for cl in content_losses:
                content_score += cl.loss

            style_score *= style_weight
            content_score *= content_weight

            loss = style_score + content_score
            loss.backward()

            run[0] += 1
            if run[0] % 50 == 0:
                print("run {}:".format(run))
                print('Style Loss : {:4f} Content Loss: {:4f}'.format(
                    style_score.item(), content_score.item()))
                print()

            return style_score + content_score
#         torch.nn.utils.clip_grad_value_(input_img, 0.1) # we had a problem with exploiding gradiants, but we fixed it without the need of clip_grad, however we have to use it at some conditions
        optimizer.step(closure)

    # a last correction...
    with torch.no_grad():
        input_img.clamp_(0, 1)

    return input_img

In [ ]:
unloader = transforms.ToPILImage()  # reconvert into PIL image

plt.ion()

def imshow(tensor, title=None):
    image = tensor.cpu().clone()  # we clone the tensor to not do changes on it
    image = image.squeeze(0)      # remove the fake batch dimension
    image = unloader(image)
    plt.imshow(image)
    if title is not None:
        plt.title(title)
    plt.pause(0.001) # pause a bit so that plots are updated

In [ ]:
# Here we can manipulate the layers to be used for the style transfer, we tried removing different layers for each style, we had an intresting results.
style_layers_default1 = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']
style_layers_default2 = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']

In [ ]:
# We have 2 styles to transfer to the content image, we start by loading them
style_img1 = image_loader("/home/faisal/Desktop/style5.jpeg")
style_img2 = image_loader("/home/faisal/Desktop/style11.jpg")
# load the content image
content_img = image_loader("/home/faisal/Desktop/content.png")
# here the input_img is the same as the content image, we will be modifying the content image later
input_img = content_img.clone()
# Here we have a parammeter called p, but in function we used 0.5 as a value, this will be resposible for the ratio between the two styles,
# for example we can get 80% of 1 style and 20% of the second one, and playing with variable will make a huge role in the output
output = run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                            content_img, style_img1, 0.5, style_img2, input_img)



In [ ]:
from PIL import Image

In [ ]:
plt.figure()
imshow(output)
# imshow(output, title='Output Image')
# img = unloader(output.detach().cpu()[0])
# img.save("output3.png")
# sphinx_gallery_thumbnail_number = 4
plt.ioff()
plt.show()

In [ ]:
# img = unloader(output.detach().cpu()[0])
# img.save("output3.png")

In [ ]:
style_img1.shape

In [ ]:


im_rgb = style_img1[:, [2, 1, 0], :, :]
# Image.fromarray(im_rgb).save('data/dst/lena_swap.jpg')

# im_rgb = im_bgr[:, :, ::-1]
# Image.fromarray(im_rgb).save('data/dst/lena_swap_2.jpg')


In [ ]:
imshow(im_rgb)

In [ ]:



plt.figure()
imshow(style_img1, title='Style Image')

plt.figure()
imshow(style_img2, title='Style Image')

plt.figure()
imshow(content_img, title='Content Image')